In [1]:
import pandas as pd

s = pd.read_csv('sample_submission.csv')
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [2]:
from shutil import copyfile
import os

path = 'images/'
train_path = 'train_image/'
test_path = 'test_image/'

os.mkdir(train_path) if not os.path.exists(train_path) else None
os.mkdir(test_path) if not os.path.exists(test_path) else None

for image in os.listdir(path):
    kind, name = image.split('_')
    if kind == 'Test':
        copyfile(path+image, test_path+name)
    else:
        copyfile(path+image, train_path+name)
    


In [15]:

for i in train['image_id']:
    print(i)

Train_0
Train_1
Train_2
Train_3
Train_4
Train_5
Train_6
Train_7
Train_8
Train_9
Train_10
Train_11
Train_12
Train_13
Train_14
Train_15
Train_16
Train_17
Train_18
Train_19
Train_20
Train_21
Train_22
Train_23
Train_24
Train_25
Train_26
Train_27
Train_28
Train_29
Train_30
Train_31
Train_32
Train_33
Train_34
Train_35
Train_36
Train_37
Train_38
Train_39
Train_40
Train_41
Train_42
Train_43
Train_44
Train_45
Train_46
Train_47
Train_48
Train_49
Train_50
Train_51
Train_52
Train_53
Train_54
Train_55
Train_56
Train_57
Train_58
Train_59
Train_60
Train_61
Train_62
Train_63
Train_64
Train_65
Train_66
Train_67
Train_68
Train_69
Train_70
Train_71
Train_72
Train_73
Train_74
Train_75
Train_76
Train_77
Train_78
Train_79
Train_80
Train_81
Train_82
Train_83
Train_84
Train_85
Train_86
Train_87
Train_88
Train_89
Train_90
Train_91
Train_92
Train_93
Train_94
Train_95
Train_96
Train_97
Train_98
Train_99
Train_100
Train_101
Train_102
Train_103
Train_104
Train_105
Train_106
Train_107
Train_108
Train_109
Train_110


In [3]:
import keras
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train = x_train/255.
x_test = x_test/255. 
num_classes = 10 
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

print(y_train)

[[0. 0. 0. ... 0. 0. 1.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [2]:
import os
import cv2
import numpy as np
import pandas as pd
from multiprocessing import Pool, cpu_count
workers = cpu_count()
executor = Pool(processes=workers)   
from PIL import Image

def letterbox_image(image, size):
    '''resize image with unchanged aspect ratio using padding'''
    iw, ih = image.size
    w, h = size
    scale = min(w/iw, h/ih)
    nw = int(iw*scale)
    nh = int(ih*scale)

    image = image.resize((nw,nh), Image.BICUBIC)
    new_image = Image.new('RGB', size, (128,128,128))
    new_image.paste(image, ((w-nw)//2, (h-nh)//2))
    return new_image

def _preprocess_data(image_path, csv_path):
    csv = pd.read_csv(csv_path)
#     print(csv.head())
    labels = True if csv.shape[1] == 5 else False
    count = 0
    for image in csv['image_id']:
        if count == 10:
            break
        path = image_path + str(image) + '.jpg'
        im = Image.open(path)
        im = np.array(letterbox_image(im, (600,600)))
        im = np.reshape(im,[1]+list(im.shape))
        if 'X' not in locals():
            X = im
        else:
            X = np.concatenate((X,im))
        count+=1
#     X = X/255
    if labels:
        y = csv.loc[:, 'healthy':].values
        return X,y
    else:
        return X
    
image_path = 'C:/Users/emage/OneDrive/Desktop/apple_k/data/train_image/'
csv_path = 'C:/Users/emage/OneDrive/Desktop/apple_k/data/train.csv'
x,y = _preprocess_data(image_path, csv_path)
# print(x.shape)

In [26]:
y = y[0:10,:]
# print(a.shape)

In [27]:
print(x.shape,y.shape)

(10, 600, 600, 3) (10, 4)


In [25]:
from keras.preprocessing.image import ImageDataGenerator

aug = ImageDataGenerator(rotation_range=20, zoom_range=0.15, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15,
                        horizontal_flip=True, fill_mode="nearest")


Using TensorFlow backend.
c:\users\emage\.conda\envs\apple\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\emage\.conda\envs\apple\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\emage\.conda\envs\apple\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\emage\.conda\envs\apple\lib\site-packages\tenso

In [41]:
a = aug.flow(x,y,batch_size=2)

In [44]:
import cv2
count=0
batch = 0
for u,v in a:
    if batch >= len(x)/2:
        break
    for i in u:
        count+=1
        cv2.imshow('w',i)
        cv2.waitKey()
        
# #     print(y)
    batch+=1
print(count)
cv2.destroyAllWindows()

10


In [11]:
import numpy as np
import imgaug.augmenters as iaa
import cv2

path = 'C:/Users/emage/OneDrive/Desktop/apple_k/data/train_image/1.jpg'
x = cv2.imread(path)
print(x.shape)
seq = iaa.Sequential([
    iaa.Crop(px=(1, 16), keep_size=False),
    iaa.Fliplr(0.5),
    iaa.GaussianBlur(sigma=(0, 3.0))
])

# for batch_idx in range(100):
#     images = load_batch(batch_idx)
images_aug = seq(images=x)
print(len(images_aug))
cv2.imshow('w',images_aug)
cv2.waitKey()
cv2.destroyAllWindows()
#     train_on_images(images_aug)

(1365, 2048, 3)
1365


TypeError: Expected Ptr<cv::UMat> for argument 'mat'

In [46]:
print(x.shape)
im = np.reshape(x,[1]+list(x.shape))
seq = iaa.Sequential([
    iaa.Affine(rotate=(-50, 50)),
    iaa.Crop(percent=(0, 0.2))], random_order=True)

image_aug = seq(images=im)
print(image_aug.shape)
cv2.namedWindow('image',cv2.WINDOW_NORMAL)
cv2.resizeWindow('image', 600,600)

cv2.imshow('image',image_aug[0])
cv2.namedWindow('image1',cv2.WINDOW_NORMAL)
cv2.resizeWindow('image1', 600,600)

cv2.imshow('image1',x)
cv2.waitKey()
cv2.destroyAllWindows()

(1365, 2048, 3)
(1, 1365, 2048, 3)


In [31]:
import imgaug as ia
import imgaug.augmenters as iaa

def augument(images, labels, number_aug = 2):
    sometimes = lambda aug: iaa.Sometimes(0.5, aug)
    seq = iaa.Sequential(
        [

            iaa.Fliplr(0.5), 
            iaa.Flipud(0.2), 

            sometimes(iaa.CropAndPad(
                percent=(-0.05, 0.1),
                pad_mode=ia.ALL,
                pad_cval=(0, 255)
            )),
            sometimes(iaa.Affine(
                scale={"x": (0.8, 1.2), "y": (0.8, 1.2)}, 
                translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)}, 
                rotate=(-45, 45), 
                shear=(-16, 16), 
                order=[0, 1], 
                cval=(0, 255), 
                mode=ia.ALL 
            )),

            iaa.SomeOf((0, 5),
                [
                    sometimes(iaa.Superpixels(p_replace=(0, 1.0), n_segments=(20, 200))), 
                    iaa.OneOf([
                        iaa.GaussianBlur((0, 3.0)), 
                        iaa.AverageBlur(k=(2, 7)), 
                        iaa.MedianBlur(k=(3, 11)), 
                    ]),
                    iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5)), 
                    iaa.Emboss(alpha=(0, 1.0), strength=(0, 2.0)), 
                    iaa.SimplexNoiseAlpha(iaa.OneOf([
                        iaa.EdgeDetect(alpha=(0.5, 1.0)),
                        iaa.DirectedEdgeDetect(alpha=(0.5, 1.0), direction=(0.0, 1.0)),
                    ])),
                    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5), 
                    iaa.OneOf([
                        iaa.Dropout((0.01, 0.1), per_channel=0.5),
                        iaa.CoarseDropout((0.03, 0.15), size_percent=(0.02, 0.05), per_channel=0.2),
                    ]),
                    iaa.Invert(0.05, per_channel=True), 
                    iaa.Add((-10, 10), per_channel=0.5), 
                    iaa.AddToHueAndSaturation((-20, 20)), 
                    iaa.OneOf([
                        iaa.Multiply((0.5, 1.5), per_channel=0.5),
                        iaa.FrequencyNoiseAlpha(
                            exponent=(-4, 0),
                            first=iaa.Multiply((0.5, 1.5), per_channel=True),
                            second=iaa.LinearContrast((0.5, 2.0))
                        )
                    ]),
                    iaa.LinearContrast((0.5, 2.0), per_channel=0.5), 
                    iaa.Grayscale(alpha=(0.0, 1.0)),
                    sometimes(iaa.ElasticTransformation(alpha=(0.5, 3.5), sigma=0.25)), 
                    sometimes(iaa.PiecewiseAffine(scale=(0.01, 0.05))), 
                    sometimes(iaa.PerspectiveTransform(scale=(0.01, 0.1)))
                ],
                random_order=True
            )
        ],
        random_order=True
    )
    temp_image = np.copy(images)
    temp_labels = np.copy(labels)
    for _ in range(number_aug):
        image_aug = seq(images=images)
        # images_aug = np.vstack(seq(images=images))
        temp_image = np.concatenate((temp_image,image_aug))
        temp_labels = np.concatenate((temp_labels,labels))
        
    return temp_image, temp_labels

In [32]:
images_aug,y_true = augument(x,y)


c:\users\emage\.conda\envs\apple\lib\site-packages\imgaug\imgaug.py:184: DeprecationWarning: Function `SimplexNoiseAlpha()` is deprecated. Use `BlendAlphaSimplexNoise` instead. SimplexNoiseAlpha is deprecated. Use BlendAlphaSimplexNoise instead. The order of parameters is the same. Parameter 'first' was renamed to 'foreground'. Parameter 'second' was renamed to 'background'.
  warn_deprecated(msg, stacklevel=3)
c:\users\emage\.conda\envs\apple\lib\site-packages\imgaug\imgaug.py:184: DeprecationWarning: Function `FrequencyNoiseAlpha()` is deprecated. Use `BlendAlphaFrequencyNoise` instead. FrequencyNoiseAlpha is deprecated. Use BlendAlphaFrequencyNoise instead. The order of parameters is the same. Parameter 'first' was renamed to 'foreground'. Parameter 'second' was renamed to 'background'.
  warn_deprecated(msg, stacklevel=3)


In [33]:
print((images_aug.shape), y_true.shape)

(30, 600, 600, 3) (30, 4)


In [35]:
for k,i in enumerate(images_aug):
#     print(i.shape)
    cv2.imshow('w',i)
    cv2.imshow('w1',x[k%10])
    cv2.waitKey()
cv2.destroyAllWindows()

In [43]:
import numpy as np

temp = np.load

In [3]:
def unison_shuffled_copies(x, y):
    assert len(x) == len(y)
    p = np.random.permutation(len(x))
    return x[p], y[p]
x,y = unison_shuffled_copies(x,y)

In [36]:
import imgaug as ia
import imgaug.augmenters as iaa
import os
import cv2
import numpy as np
import pandas as pd
from PIL import Image

from keras.preprocessing.image import ImageDataGenerator




import imgaug as ia
import imgaug.augmenters as iaa

def augument(images, labels, number_aug = 2):
    sometimes = lambda aug: iaa.Sometimes(0.5, aug)
    seq = iaa.Sequential(
        [

            iaa.Fliplr(0.5), 
            iaa.Flipud(0.2), 

            sometimes(iaa.CropAndPad(
                percent=(-0.05, 0.1),
                pad_mode=ia.ALL,
                pad_cval=(0, 255)
            )),
            sometimes(iaa.Affine(
                scale={"x": (0.8, 1.2), "y": (0.8, 1.2)}, 
                translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)}, 
                rotate=(-45, 45), 
                shear=(-16, 16), 
                order=[0, 1], 
                cval=(0, 255), 
                mode=ia.ALL 
            )),

            iaa.SomeOf((0, 5),
                [
                    sometimes(iaa.Superpixels(p_replace=(0, 1.0), n_segments=(20, 200))), 
                    iaa.OneOf([
                        iaa.GaussianBlur((0, 3.0)), 
                        iaa.AverageBlur(k=(2, 7)), 
                        iaa.MedianBlur(k=(3, 11)), 
                    ]),
                    iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5)), 
                    iaa.Emboss(alpha=(0, 1.0), strength=(0, 2.0)), 
                    iaa.SimplexNoiseAlpha(iaa.OneOf([
                        iaa.EdgeDetect(alpha=(0.5, 1.0)),
                        iaa.DirectedEdgeDetect(alpha=(0.5, 1.0), direction=(0.0, 1.0)),
                    ])),
                    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5), 
                    iaa.OneOf([
                        iaa.Dropout((0.01, 0.1), per_channel=0.5),
                        iaa.CoarseDropout((0.03, 0.15), size_percent=(0.02, 0.05), per_channel=0.2),
                    ]),
                    iaa.Invert(0.05, per_channel=True), 
                    iaa.Add((-10, 10), per_channel=0.5), 
                    iaa.AddToHueAndSaturation((-20, 20)), 
                    iaa.OneOf([
                        iaa.Multiply((0.5, 1.5), per_channel=0.5),
                        iaa.FrequencyNoiseAlpha(
                            exponent=(-4, 0),
                            first=iaa.Multiply((0.5, 1.5), per_channel=True),
                            second=iaa.LinearContrast((0.5, 2.0))
                        )
                    ]),
                    iaa.LinearContrast((0.5, 2.0), per_channel=0.5), 
                    iaa.Grayscale(alpha=(0.0, 1.0)),
                    sometimes(iaa.ElasticTransformation(alpha=(0.5, 3.5), sigma=0.25)), 
                    sometimes(iaa.PiecewiseAffine(scale=(0.01, 0.05))), 
                    sometimes(iaa.PerspectiveTransform(scale=(0.01, 0.1)))
                ],
                random_order=True
            )
        ],
        random_order=True
    )
    temp_image = np.copy(images)
    temp_labels = np.copy(labels)
    for _ in range(number_aug):
        image_aug = seq(images=images)
        # images_aug = np.vstack(seq(images=images))
        temp_image = np.concatenate((temp_image,image_aug))
        temp_labels = np.concatenate((temp_labels,labels))
        
    return temp_image, temp_labels

def letterbox_image(image, size):
    iw, ih = image.size
    w, h = size
    scale = min(w/iw, h/ih)
    nw = int(iw*scale)
    nh = int(ih*scale)

    image = image.resize((nw,nh), Image.BICUBIC)
    new_image = Image.new('RGB', size, (128,128,128))
    new_image.paste(image, ((w-nw)//2, (h-nh)//2))
    return new_image

def readAndProcess(image_path):
    path = image_path
    im = Image.open(path)
    im = np.array(letterbox_image(im, (600,600)))
    # im = np.reshape(im,[1]+list(im.shape))
    return im 
    

def multithread_preprocess_data(image_path, csv_path, executor):
    csv = pd.read_csv(csv_path)
    labels = True if csv.shape[1] == 5 else False
    path = []
    for image in csv['image_id']:
        path.append(image_path + image + '.jpg')
    X = np.vstack(executor.map(readAndProcess,path))
    if labels:
        y = csv.loc[:, 'healthy':].values
        return X,y
    else:
        return X

def unison_shuffled_copies(x, y):
    assert len(x) == len(y)
    np.random.seed(2000)
    p = np.random.permutation(len(x))
    return x[p], y[p]
    
def train_generator(image_dir, train, batch_size, y):
    i = 0
    while True:
        image_data = []
        box_data = []
        base = i

        # only use for train
        batch_size /=3

        for b in range(int(batch_size)):
            image = readAndProcess(image_dir + str(train[base+b]) + '.jpg')

            image_data.append(image)

            i += 1

        image_data = np.array(image_data)
        y_true = y[base:i]
        image_data, y_true = augument(image_data, y_true)
        yield [image_data, y_true]


def val_generator(image_dir, train, batch_size, y):
    i = 0
    while True:
        image_data = []
        box_data = []
        base = i
        for b in range(int(batch_size)):
            image = readAndProcess(image_dir + str(train[base+b]) + '.jpg')
            image_data.append(image)
            i += 1 
        image_data = np.array(image_data)
        y_true = y[base:i]
        # print(image_data.shape, y_true.shape,i)
        yield [image_data, y_true]

In [37]:
csv = pd.read_csv('train.csv')
image_named = np.array(csv['image_id'])
y = csv.loc[:, 'healthy':].values
image_named , y = unison_shuffled_copies(image_named, y)
image_named = list(image_named)

num_train = 1500
train = image_named[:num_train]
val = image_named[num_train:]

In [372]:
def train_generator(image_dir, train, batch_size, y):
    i = 0
    batch_size = batch_size/3

    while True :
        image_data = []
        base = 0
        # only use for train
#         print(int(batch_size))
        for b in range(int(batch_size)):
#             print(b)
            image = readAndProcess(image_dir + str(train[base+b]) + '.jpg')
            image_data.append(image)
            i = (i+1)
#         print(i)
#         print(base)
        image_data = np.array(image_data)
        y_true = y[base:i]
        image_data, y_true = augument(image_data, y_true)
        yield [image_data, *y_true]
#         yield i
        
a = train_generator('train_image/',train, 3, y)

In [371]:
for i in a:
    break

6


In [293]:
def data_generator():
    '''data generator for fit_generator'''
    i = 0
    while True:
        image_data = []
        box_data = []
        print(3)
        for b in range(3):
#             if i==0:
#                 np.random.shuffle(annotation_lines)
#             image, box = get_random_data(annotation_lines[i], input_shape, random=True)
#             image_data.append(image)
#             box_data.append(box)
            i = (i+1) 
        print(i)
        yield i
with open('train.txt') as f:
    lines = f.readlines()
anchors = get_anchors('yolo_anchors.txt')

np.random.seed(100)
np.random.shuffle(lines)
num_train = len(lines)
t = data_generator()

In [310]:
for i in t:
    break

3
51


In [65]:
from functools import reduce

from PIL import Image
import numpy as np
from matplotlib.colors import rgb_to_hsv, hsv_to_rgb
def rand(a=0, b=1):
    return np.random.rand()*(b-a) + a

def get_random_data(annotation_line, input_shape, random=True, max_boxes=20, jitter=.3, hue=.1, sat=1.5, val=1.5, proc_img=True):
    '''random preprocessing for real-time data augmentation'''
    line = annotation_line.split()
    image = Image.open(line[0])
    iw, ih = image.size
    h, w = input_shape
    box = np.array([np.array(list(map(int,box.split(',')))) for box in line[1:]])

    if not random:
        # resize image
        scale = min(w/iw, h/ih)
        nw = int(iw*scale)
        nh = int(ih*scale)
        dx = (w-nw)//2
        dy = (h-nh)//2
        image_data=0
        if proc_img:
            image = image.resize((nw,nh), Image.BICUBIC)
            new_image = Image.new('RGB', (w,h), (128,128,128))
            new_image.paste(image, (dx, dy))
            image_data = np.array(new_image)/255.

        # correct boxes
        box_data = np.zeros((max_boxes,5))
        if len(box)>0:
            np.random.shuffle(box)
            if len(box)>max_boxes: box = box[:max_boxes]
            box[:, [0,2]] = box[:, [0,2]]*scale + dx
            box[:, [1,3]] = box[:, [1,3]]*scale + dy
            box_data[:len(box)] = box

        return image_data, box_data

    # resize image
    new_ar = w/h * rand(1-jitter,1+jitter)/rand(1-jitter,1+jitter)
    scale = rand(.25, 2)
    if new_ar < 1:
        nh = int(scale*h)
        nw = int(nh*new_ar)
    else:
        nw = int(scale*w)
        nh = int(nw/new_ar)
    image = image.resize((nw,nh), Image.BICUBIC)

    # place image
    dx = int(rand(0, w-nw))
    dy = int(rand(0, h-nh))
    new_image = Image.new('RGB', (w,h), (128,128,128))
    new_image.paste(image, (dx, dy))
    image = new_image

    # flip image or not
    flip = rand()<.5
    if flip: image = image.transpose(Image.FLIP_LEFT_RIGHT)

    # distort image
    hue = rand(-hue, hue)
    sat = rand(1, sat) if rand()<.5 else 1/rand(1, sat)
    val = rand(1, val) if rand()<.5 else 1/rand(1, val)
    x = rgb_to_hsv(np.array(image)/255.)
    x[..., 0] += hue
    x[..., 0][x[..., 0]>1] -= 1
    x[..., 0][x[..., 0]<0] += 1
    x[..., 1] *= sat
    x[..., 2] *= val
    x[x>1] = 1
    x[x<0] = 0
    image_data = hsv_to_rgb(x) # numpy array, 0 to 1

    # correct boxes
    box_data = np.zeros((max_boxes,5))
    if len(box)>0:
        np.random.shuffle(box)
        box[:, [0,2]] = box[:, [0,2]]*nw/iw + dx
        box[:, [1,3]] = box[:, [1,3]]*nh/ih + dy
        if flip: box[:, [0,2]] = w - box[:, [2,0]]
        box[:, 0:2][box[:, 0:2]<0] = 0
        box[:, 2][box[:, 2]>w] = w
        box[:, 3][box[:, 3]>h] = h
        box_w = box[:, 2] - box[:, 0]
        box_h = box[:, 3] - box[:, 1]
        box = box[np.logical_and(box_w>1, box_h>1)] # discard invalid box
        if len(box)>max_boxes: box = box[:max_boxes]
        box_data[:len(box)] = box

    return image_data, box_data

def preprocess_true_boxes(true_boxes, input_shape, anchors, num_classes):
    '''Preprocess true boxes to training input format

    Parameters
    ----------
    true_boxes: array, shape=(m, T, 5)
        Absolute x_min, y_min, x_max, y_max, class_id relative to input_shape.
    input_shape: array-like, hw, multiples of 32
    anchors: array, shape=(N, 2), wh
    num_classes: integer

    Returns
    -------
    y_true: list of array, shape like yolo_outputs, xywh are reletive value

    '''
    assert (true_boxes[..., 4]<num_classes).all(), 'class id must be less than num_classes'
    num_layers = len(anchors)//3 # default setting
    anchor_mask = [[6,7,8], [3,4,5], [0,1,2]] if num_layers==3 else [[3,4,5], [1,2,3]]

    true_boxes = np.array(true_boxes, dtype='float32')
    input_shape = np.array(input_shape, dtype='int32')
    boxes_xy = (true_boxes[..., 0:2] + true_boxes[..., 2:4]) // 2
    boxes_wh = true_boxes[..., 2:4] - true_boxes[..., 0:2]
    true_boxes[..., 0:2] = boxes_xy/input_shape[::-1]
    true_boxes[..., 2:4] = boxes_wh/input_shape[::-1]

    m = true_boxes.shape[0]
    grid_shapes = [input_shape//{0:32, 1:16, 2:8}[l] for l in range(num_layers)]
    y_true = [np.zeros((m,grid_shapes[l][0],grid_shapes[l][1],len(anchor_mask[l]),5+num_classes),
        dtype='float32') for l in range(num_layers)]

    # Expand dim to apply broadcasting.
    anchors = np.expand_dims(anchors, 0)
    anchor_maxes = anchors / 2.
    anchor_mins = -anchor_maxes
    valid_mask = boxes_wh[..., 0]>0

    for b in range(m):
        # Discard zero rows.
        wh = boxes_wh[b, valid_mask[b]]
        if len(wh)==0: continue
        # Expand dim to apply broadcasting.
        wh = np.expand_dims(wh, -2)
        box_maxes = wh / 2.
        box_mins = -box_maxes

        intersect_mins = np.maximum(box_mins, anchor_mins)
        intersect_maxes = np.minimum(box_maxes, anchor_maxes)
        intersect_wh = np.maximum(intersect_maxes - intersect_mins, 0.)
        intersect_area = intersect_wh[..., 0] * intersect_wh[..., 1]
        box_area = wh[..., 0] * wh[..., 1]
        anchor_area = anchors[..., 0] * anchors[..., 1]
        iou = intersect_area / (box_area + anchor_area - intersect_area)

        # Find best anchor for each true box
        best_anchor = np.argmax(iou, axis=-1)

        for t, n in enumerate(best_anchor):
            for l in range(num_layers):
                if n in anchor_mask[l]:
                    i = np.floor(true_boxes[b,t,0]*grid_shapes[l][1]).astype('int32')
                    j = np.floor(true_boxes[b,t,1]*grid_shapes[l][0]).astype('int32')
                    k = anchor_mask[l].index(n)
                    c = true_boxes[b,t, 4].astype('int32')
                    y_true[l][b, j, i, k, 0:4] = true_boxes[b,t, 0:4]
                    y_true[l][b, j, i, k, 4] = 1
                    y_true[l][b, j, i, k, 5+c] = 1

    return y_true
def get_anchors(anchors_path):
    '''loads the anchors from a file'''
    with open(anchors_path) as f:
        anchors = f.readline()
    anchors = [float(x) for x in anchors.split(',')]
    return np.array(anchors).reshape(-1, 2)

In [13]:
import numpy as np
def unison_shuffled_copies(x, y):
    assert len(x) == len(y)
    np.random.seed(2000)
    p = np.random.permutation(len(x))
    return x[p], y[p]
image_named = np.array(train['image_id'])
y = train.loc[:, 'healthy':].values
print(y[:10])
# image_named , y = unison_shuffled_copies(image_named, y)
# image_named = list(image_named)
# num_train = 1500
# train = image_named[:num_train]
# val = image_named[num_train:]
# y_train = np.copy(y[:num_train])
# y_val = np.copy(y[num_train:])

[[0 0 0 1]
 [0 1 0 0]
 [1 0 0 0]
 [0 0 1 0]
 [1 0 0 0]
 [1 0 0 0]
 [0 1 0 0]
 [0 0 0 1]
 [0 0 0 1]
 [1 0 0 0]]


In [9]:
from sklearn.model_selection import train_test_split
x_train ,x_val, y_train,y_val = train_test_split(image_named, y, test_size = 0.22, random_state = 42, shuffle = True)
print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)

(1420,)
(1420, 4)
(401,)
(401, 4)


In [10]:
print(np.sum(y[:,0]))
print(np.sum(y[:,1]))
print(np.sum(y[:,2]))
print(np.sum(y[:,3]))

516
91
622
592


In [11]:
print(np.sum(y_train[:,0]))
print(np.sum(y_train[:,1]))
print(np.sum(y_train[:,2]))
print(np.sum(y_train[:,3]))
# print(np.sum(y_train,axis=0))

404
72
488
456


In [12]:
print(np.sum(y_val[:,0]))
print(np.sum(y_val[:,1]))
print(np.sum(y_val[:,2]))
print(np.sum(y_val[:,3]))

112
19
134
136


In [58]:
from collections import Counter
def split_data(x,y, train_ratio = 0.8):
    train = []
    val = []
    y_train = []
    y_val = []
    num_class = y.shape[1]
    for i in range(num_class):
        num_labels = np.sum(y[:,i])
        num_train = np.int(num_labels * train_ratio)
        temp_x = x[np.where(y[:,i]==1)]
        new = np.random.choice(temp_x, num_train, replace = False)
        
#         label_train = np.zeros((num_train,num_class))
#         label_val = np.zeros((num_val,num_class))
        
    
    
    
    
split_data(image_named,y)

In [35]:
import numpy as np
import imgaug as ia
import imgaug.augmenters as iaa

def augument(images, labels, number_aug = 2):
    seq = iaa.Sequential([
    iaa.Affine(rotate=(-50, 50)),
    iaa.Crop(percent=(0, 0.2))], random_order=True)

    temp_image = np.copy(images)
    temp_labels = np.copy(labels)
    for _ in range(number_aug):
        image_aug = seq(images=images)
        # images_aug = np.vstack(seq(images=images))
        temp_image = np.concatenate((temp_image,image_aug))
        temp_labels = np.concatenate((temp_labels,labels))
        
    return temp_image, temp_labels

import cv2
path = 'C:/Users/emage/OneDrive/Desktop/tom_jerry.png'
image = cv2.imread(path)

# image = np.reshape(image, [1]+list(image.shape))
image = [image,image,image]
im, l = augument(image, np.array([0,0,1,0]))

In [37]:
from matplotlib import pyplot as plt
cv2.namedWindow('w', cv2.WINDOW_NORMAL)
cv2.resizeWindow('w', 1200,600)

print(im.shape)
for k,i in enumerate(im):
    print(k)
#     cv2.imshow('w',i)
#     cv2.waitKey()
# cv2.destroyAllWindows()

(9, 1080, 1746, 3)
0
1
2
3
4
5
6
7
8


In [48]:
import pickle
path = r'C:\Users\emage\OneDrive\Desktop\apple_k\resnet\log\imageNet\non_letter_image_Relu\history'

a = pickle.load(open(path,'rb'))



{'val_loss': [248.19297790527344, 21.205278396606445, 5.3035359382629395, 2.0561983585357666, 2.8733646869659424, 3.356678009033203, 6.0684967041015625, 2.391111373901367, 2.6583218574523926, 1.2787601947784424, 1.4704539775848389, 0.49122923612594604, 0.27882248163223267, 0.7168084383010864, 0.9245395064353943, 0.196638286113739, 0.45205578207969666, 0.18343055248260498, 0.3750162124633789, 0.3625688850879669], 'val_accuracy': [0.33250001072883606, 0.2800000011920929, 0.2800000011920929, 0.3375000059604645, 0.2800000011920929, 0.2800000011920929, 0.2775000035762787, 0.2775000035762787, 0.41499999165534973, 0.5425000190734863, 0.5874999761581421, 0.8424999713897705, 0.8075000047683716, 0.7900000214576721, 0.6924999952316284, 0.8125, 0.8650000095367432, 0.8774999976158142, 0.8824999928474426, 0.8999999761581421], 'loss': [1.2721070786091415, 0.9031198651275851, 0.7574827125803991, 0.6903658400035717, 0.6693089352497323, 0.5772549294951287, 0.48787951264107093, 0.5417735053395684, 0.4274

In [96]:
import plotly.graph_objects as go

val_loss = np.round(a['val_loss'],3)
train_loss = np.round(a['loss'],3)
epoch = np.arange(1,len(val_loss)+1)

fig = go.Figure()

fig.add_trace(go.Scatter(x = epoch, y = val_loss, mode = 'lines+markers', name = 'fig1'))

fig.add_trace(go.Scatter(x = epoch, y = train_loss, mode = 'lines+markers', name = 'fig2'))

fig.update_layout(title = 'Non_letter_image_Relu training process',
                 xaxis_title = 'epoch',
                 yaxis_title = 'loss')
fig.show()
fig.write_image("fig.png")


ValueError: Image generation requires the psutil package.

Install using pip:
    $ pip install psutil

Install using conda:
    $ conda install psutil


In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd

image_path = 'C:/Users/ADMINS/Desktop/apple_k/data/train_image/'
labels_path = 'C:/Users/ADMINS/Desktop/apple_k/data/train.csv'
csv = pd.read_csv(labels_path)
image_named = np.array(csv['image_id'])
y = csv.loc[:, 'healthy':].values

train ,val, y_train,y_val = train_test_split(image_named, y, test_size = 0.22, random_state = 42, shuffle = True)

In [19]:
def augument(images, labels, number_aug = 2):
#     seq = iaa.Sequential([
#     iaa.Affine(rotate=(-50, 50)),
#     iaa.Crop(percent=(0, 0.2))], random_order=True)

    temp_image = np.copy(images)
    temp_labels = np.copy(labels)
    for _ in range(number_aug):
#         image_aug = seq(images=images)
        temp_image = np.concatenate((temp_image,images))
        temp_labels = np.concatenate((temp_labels,labels))
        
    return temp_image, temp_labels
def train_generator(image_dir, train, batch_size, y, num_train):
    i = 0
    batch_size = batch_size//3
    while True :
        image_data = []
        base = i
        # only use for train
        for b in range(int(batch_size)):
            num = (base+b)%num_train
#             image = readAndProcess(image_dir + str(train[num]) + '.jpg')
            image_data.append(image_dir + str(train[num]) + '.jpg')
            i = (i+1)%num_train
        image_data = np.array(image_data)
        y_true = np.copy(y[base:i])
        if len(y_true) != len(image_data):
            y_true = np.copy(y[base:len(y)])
            y_true = np.concatenate((y_true,np.copy(y[:i]))) 
        image_data, y_true = augument(image_data, y_true)
#         image_data = image_data/255
        yield [image_data, y_true]

In [21]:
a = train_generator(image_path, train, 6, y_train, 100)
count = 0
for i in a:
    print(i)
    count+=1
    if count == 10:
        break

[array(['C:/Users/ADMINS/Desktop/apple_k/data/train_image/764.jpg',
       'C:/Users/ADMINS/Desktop/apple_k/data/train_image/433.jpg',
       'C:/Users/ADMINS/Desktop/apple_k/data/train_image/764.jpg',
       'C:/Users/ADMINS/Desktop/apple_k/data/train_image/433.jpg',
       'C:/Users/ADMINS/Desktop/apple_k/data/train_image/764.jpg',
       'C:/Users/ADMINS/Desktop/apple_k/data/train_image/433.jpg'],
      dtype='<U56'), array([[0, 0, 1, 0],
       [1, 0, 0, 0],
       [0, 0, 1, 0],
       [1, 0, 0, 0],
       [0, 0, 1, 0],
       [1, 0, 0, 0]], dtype=int64)]
[array(['C:/Users/ADMINS/Desktop/apple_k/data/train_image/199.jpg',
       'C:/Users/ADMINS/Desktop/apple_k/data/train_image/745.jpg',
       'C:/Users/ADMINS/Desktop/apple_k/data/train_image/199.jpg',
       'C:/Users/ADMINS/Desktop/apple_k/data/train_image/745.jpg',
       'C:/Users/ADMINS/Desktop/apple_k/data/train_image/199.jpg',
       'C:/Users/ADMINS/Desktop/apple_k/data/train_image/745.jpg'],
      dtype='<U56'), array([[1